In [ ]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
load_dotenv("deepseek_api.env", override=True)

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [ ]:
# build a AI chatbot with conversation history

model = init_chat_model(model="deepseek-chat", model_provider="deepseek")
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are hoshiro, is a helpful assistant."),
    ("human", "{input}")
])

qa_chain = prompt | model | parser

In [ ]:
async def chat_response(message, history):
    partial_message = ""

    async for chunk in qa_chain.astream({"input": message}):
        partial_message += chunk
        yield partial_message


def create_chatbot():

    css = """
    .main-container { max-width: 1200px; margin: 0 auto; padding: 20px; }
    .header-text { text-align: center; margin-bottom: 20px; }
    """

    with gr.Blocks(title="DeepSeek Chat", css=css) as demo:
        with gr.Column(elem_classes=["main-container"]):
            gr.Markdown("# LangChain", elem_classes=["header-text"])
            gr.Markdown("**AI Chatbot based on LangChain", elem_classes=["header-text"])

            # chatbox
            chatbot = gr.Chatbot(
                height=500,
                show_copy_button=True,
                avatar_images=(
                    # user avatar, bot avatar
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@14.0.2/assets/72x72/1f9d1-200d-1f4bb.png",
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@14.0.2/assets/72x72/1f916.png",
                ),
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Please input your question",
                    container=False,
                    scale=7,
                )
                submit = gr.Button("send", scale=1, variant="primary")
                clear = gr.Button("clear", scale=1)

            # handling messages and update stream
            async def respond(message, chat_history):
                if not message or not message.strip():
                    # clean input
                    yield "", chat_history
                    return

                # 1) add the user's message to the history and display it
                chat_history = (chat_history or []) + [(message, None)]
                yield "", chat_history

                # 2) stream the ai reply
                async for partial in chat_response(message, chat_history):
                    chat_history[-1] = (message, partial)
                    yield "", chat_history

            # clean conversation history
            def clear_history():
                return [], ""

            # bind events: submit on enter, click to send, click to clear
            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            submit.click(respond, [msg, chatbot], [msg, chatbot])
            clear.click(clear_history, outputs=[chatbot, msg])

    return demo

demo = create_chatbot()
demo.launch(
    server_name="0.0.0.0",
    server_port=7859,
    share=False,
    debug=True
)